In [7]:
# read docs

docs_path = "/home/nikita/PycharmProjects/ml-diplom/ir/data/data/docs.tsv"
queries_path = "/home/nikita/PycharmProjects/ml-diplom/ir/data/data/queries.numerate.txt"

In [12]:
def read_docs_tsv(path:str) -> list:
    result = []
    with open(path) as f:
        for line in f.readlines():
            idx, text = line.split(maxsplit=1)
            result.append((str(idx), text.split()))
    return result

def read_file(path:str) -> list:
    result = []
    with open(path) as f:
        for line in f.readlines():
            result.append(line)
    return result

In [13]:
docs = read_docs_tsv(docs_path)
queries = read_file(queries_path)

In [19]:
def build_inverted_index():
    inverted_index = {}
    for (doc_idx, doc) in docs:
        for word in doc:
            if word not in inverted_index:    
                inverted_index[word] = set()
            inverted_index[word].add(doc_idx)
    return inverted_index

reversed_indexes = build_inverted_index()

In [20]:
class QueryTree:
    def __init__(self, qid:int, query:list):
        self.qid = qid
        self.query = query
    
    def execute_query(self, reversed_index: dict) -> set:
        result_set = None
        for sub_query in self.query:
            merged_set = set()
            for synonym in sub_query:    
                current_set = set() if synonym not in reversed_index else reversed_index[synonym]
                merged_set = merged_set | current_set
            if result_set is None:
                result_set = merged_set
            result_set = result_set & merged_set
        return result_set

def prepare_query(query:str) -> QueryTree: 
    def inner(x:str) -> list:
        if x[0] == '(':
            x = x[1:-1]
            return x.split('|')
        return [x]
    split_values = query.split()
    idx = split_values[0]
    parsed_query = list(map(inner, split_values[1:]))
    return QueryTree(idx, parsed_query)

In [21]:
queries_tree = list(map(prepare_query, queries))

In [24]:
rs = queries_tree[44].execute_query(reversed_indexes)

"""
top = []
for k, v in res.items():
    top.append((k, len(v)))

top.sort(key=lambda x:x[1], reverse=True)
"""

'\ntop = []\nfor k, v in res.items():\n    top.append((k, len(v)))\n\ntop.sort(key=lambda x:x[1], reverse=True)\n'